In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

29986


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数
29981,2021-10-10,47,沖縄県,14,49879,0,341,10.39
29982,2021-10-11,47,沖縄県,7,49886,4,345,10.25
29983,2021-10-12,47,沖縄県,29,49915,0,345,9.64
29984,2021-10-13,47,沖縄県,32,49947,5,350,9.77
29985,2021-10-14,47,沖縄県,33,49980,6,356,10.94


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数
42,静岡県,14036,26701,209,5127.13
43,香川県,23606,4693,38,3434.75
44,高知県,24882,4126,31,4132.08
45,鳥取県,19778,1658,5,2081.87
46,鹿児島県,29348,9088,65,3967.03


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数
29979,2021-10-08,47,沖縄県,29,49850,4,336,13.35,194.0
29980,2021-10-09,47,沖縄県,15,49865,5,341,11.42,166.0
29981,2021-10-10,47,沖縄県,14,49879,0,341,10.39,151.0
29982,2021-10-11,47,沖縄県,7,49886,4,345,10.25,149.0
29983,2021-10-12,47,沖縄県,29,49915,0,345,9.64,140.0
29984,2021-10-13,47,沖縄県,32,49947,5,350,9.77,142.0
29985,2021-10-14,47,沖縄県,33,49980,6,356,10.94,159.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
29976,2021-10-05,47,沖縄県,38,49775,4,327,20.17,293.0,3.222222,0.290636,-0.333333
29977,2021-10-06,47,沖縄県,30,49805,0,327,17.48,254.0,-0.210526,0.230486,-0.565217
29978,2021-10-07,47,沖縄県,16,49821,5,332,14.18,206.0,-0.466667,0.174171,-0.750000
29979,2021-10-08,47,沖縄県,29,49850,4,336,13.35,194.0,0.812500,0.341582,-0.292683
29980,2021-10-09,47,沖縄県,15,49865,5,341,11.42,166.0,-0.482759,0.265648,-0.651163
29981,2021-10-10,47,沖縄県,14,49879,0,341,10.39,151.0,-0.066667,0.302636,-0.517241
29982,2021-10-11,47,沖縄県,7,49886,4,345,10.25,149.0,-0.500000,0.329729,-0.222222
29983,2021-10-12,47,沖縄県,29,49915,0,345,9.64,140.0,3.142857,0.318391,-0.236842
29984,2021-10-13,47,沖縄県,32,49947,5,350,9.77,142.0,0.103448,0.363245,0.066667
29985,2021-10-14,47,沖縄県,33,49980,6,356,10.94,159.0,0.031250,0.434376,1.062500


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
637,2021-10-14,北海道,45,0,160.0,1.142857,0.249675,2.214286
1275,2021-10-14,青森県,8,0,101.0,-0.764706,NaN,1.000000
1913,2021-10-14,岩手県,0,0,1.0,NaN,NaN,NaN
2551,2021-10-14,宮城県,3,1,14.0,inf,NaN,-0.250000
3189,2021-10-14,秋田県,0,0,2.0,NaN,NaN,-1.000000


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,15312,14725,163,5781.26,2021-10-14,5,0,48.0,-0.285714,-0.042517,-0.666667
1,京都府,16588,35766,289,9681.17,2021-10-14,9,0,120.0,-0.571429,-0.007986,-0.666667
2,佐賀県,26158,5805,29,4981.02,2021-10-14,0,0,20.0,-1.000000,-0.314286,-1.000000
3,兵庫県,17864,78084,1392,9986.67,2021-10-14,30,0,264.0,-0.230769,0.241938,-0.655172
4,北海道,638,60519,1469,8058.90,2021-10-14,45,0,160.0,1.142857,0.249675,2.214286


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,15312,14725,163,5781.26,2021-10-14,5,0,48.0,-0.285714,-0.042517,-0.666667,1813859
1,京都府,16588,35766,289,9681.17,2021-10-14,9,0,120.0,-0.571429,-0.007986,-0.666667,2545899
2,佐賀県,26158,5805,29,4981.02,2021-10-14,0,0,20.0,-1.000000,-0.314286,-1.000000,823810
3,兵庫県,17864,78084,1392,9986.67,2021-10-14,30,0,264.0,-0.230769,0.241938,-0.655172,5549568
4,北海道,638,60519,1469,8058.90,2021-10-14,45,0,160.0,1.142857,0.249675,2.214286,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,638,60519,1469,8058.90,2021-10-14,45,0,160.0,1.142857,0.249675,2.214286,5267762,114.885600,1148.856004,3.037343
41,青森県,1276,5847,38,3254.60,2021-10-14,8,0,101.0,-0.764706,NaN,1.000000,1275783,45.830678,458.306781,7.916707
19,岩手県,1914,3486,53,1988.26,2021-10-14,0,0,1.0,NaN,NaN,NaN,1235517,28.214909,282.149092,0.080938
11,宮城県,2552,16249,118,4930.56,2021-10-14,3,1,14.0,inf,NaN,-0.250000,2292385,70.882509,708.825088,0.610718
36,秋田県,3190,1893,27,1371.38,2021-10-14,0,0,2.0,NaN,NaN,-1.000000,985416,19.210161,192.101610,0.202960
15,山形県,3828,3524,56,2281.76,2021-10-14,5,0,14.0,-0.285714,NaN,inf,1082296,32.560409,325.604086,1.293546
35,福島県,4466,9470,175,3589.35,2021-10-14,1,0,7.0,-0.750000,NaN,-0.833333,1881981,50.319318,503.193178,0.371948
38,茨城県,5104,24337,217,5950.42,2021-10-14,5,0,73.0,-0.545455,0.098917,-0.687500,2921436,83.304923,833.049226,2.498771
27,栃木県,5742,15404,116,5567.54,2021-10-14,5,0,60.0,-0.285714,0.479600,-0.705882,1965516,78.371278,783.712776,3.052634
37,群馬県,6380,16682,174,6005.03,2021-10-14,3,0,54.0,-0.785714,0.072336,-0.666667,1969439,84.704324,847.043244,2.741898


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      36350.425532
std       67170.093591
min        1637.000000
25%        5288.500000
50%        9470.000000
75%       25519.000000
max      377036.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,8294,377036,3069,18948.86,2021-10-14,62,12,540.0,-0.138889,-0.074969,-0.566434,13834925,272.524788,2725.247878,3.903165
1,大阪府,17226,201541,3018,15990.33,2021-10-14,112,3,784.0,-0.104000,0.039732,-0.321212,8849635,227.739336,2277.393361,8.859122
2,神奈川県,8932,168585,1294,12818.09,2021-10-14,52,4,397.0,0.040000,-0.068112,-0.490196,9209442,183.056693,1830.566933,4.310793
3,埼玉県,7018,115368,1021,10977.57,2021-10-14,35,3,222.0,-0.313725,0.038736,-0.485294,7390054,156.112526,1561.125264,3.004038
4,愛知県,14674,106259,1152,9839.92,2021-10-14,35,0,245.0,-0.125000,0.043721,-0.426230,7575530,140.266094,1402.660936,3.234097
5,千葉県,7656,100127,1011,11188.92,2021-10-14,19,1,208.0,-0.472222,-0.080297,-0.558140,6319772,158.434513,1584.345131,3.291258
6,兵庫県,17864,78084,1392,9986.67,2021-10-14,30,0,264.0,-0.230769,0.241938,-0.655172,5549568,140.702844,1407.028439,4.757127
7,福岡県,25520,74255,619,10175.36,2021-10-14,25,1,140.0,0.136364,0.162546,0.666667,5129841,144.751075,1447.510751,2.729129
8,北海道,638,60519,1469,8058.90,2021-10-14,45,0,160.0,1.142857,0.249675,2.214286,5267762,114.885600,1148.856004,3.037343
9,沖縄県,29986,49980,356,24041.48,2021-10-14,33,6,159.0,0.031250,0.434376,1.062500,1481547,337.350081,3373.500807,10.732025


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      87.806542
std       65.094476
min       19.210161
25%       45.332199
50%       70.882509
75%      103.204395
max      337.350081
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,29986,49980,356,24041.48,2021-10-14,33,6,159.0,0.031250,0.434376,1.062500,1481547,337.350081,3373.500807,10.732025
9,大阪府,17226,201541,3018,15990.33,2021-10-14,112,3,784.0,-0.104000,0.039732,-0.321212,8849635,227.739336,2277.393361,8.859122
41,青森県,1276,5847,38,3254.60,2021-10-14,8,0,101.0,-0.764706,NaN,1.000000,1275783,45.830678,458.306781,7.916707
17,岐阜県,13398,18657,216,6556.41,2021-10-14,12,0,118.0,-0.333333,0.054717,-0.520000,2032490,91.793810,917.938096,5.805687
10,奈良県,18502,15517,149,8148.22,2021-10-14,19,0,75.0,0.187500,0.434629,0.900000,1353837,114.614979,1146.149795,5.539810
23,愛媛県,24244,5294,82,2753.78,2021-10-14,7,0,68.0,0.000000,0.003721,-0.125000,1369131,38.666862,386.668624,4.966654
3,兵庫県,17864,78084,1392,9986.67,2021-10-14,30,0,264.0,-0.230769,0.241938,-0.655172,5549568,140.702844,1407.028439,4.757127
1,京都府,16588,35766,289,9681.17,2021-10-14,9,0,120.0,-0.571429,-0.007986,-0.666667,2545899,140.484756,1404.847561,4.713463
32,神奈川県,8932,168585,1294,12818.09,2021-10-14,52,4,397.0,0.040000,-0.068112,-0.490196,9209442,183.056693,1830.566933,4.310793
26,東京都,8294,377036,3069,18948.86,2021-10-14,62,12,540.0,-0.138889,-0.074969,-0.566434,13834925,272.524788,2725.247878,3.903165


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    30.000000
mean      0.223200
std       0.396999
min      -0.314286
25%      -0.005059
50%       0.085626
75%       0.344272
max       1.723810
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
14,山口県,22330,5670,92,2912.98,2021-10-14,1,1,51.0,-0.900000,1.723810,-0.666667,1369882,41.390426,413.904263,3.722948
31,石川県,10846,7928,129,4871.15,2021-10-14,2,0,17.0,-0.714286,0.826531,1.000000,1139612,69.567537,695.675370,1.491736
29,滋賀県,15950,12337,102,6102.82,2021-10-14,6,0,23.0,5.000000,0.814286,0.200000,1420948,86.822319,868.223186,1.618638
39,長崎県,26796,6056,73,3189.26,2021-10-14,4,0,34.0,-0.200000,0.728571,0.333333,1350769,44.833721,448.337206,2.517085
27,栃木県,5742,15404,116,5567.54,2021-10-14,5,0,60.0,-0.285714,0.479600,-0.705882,1965516,78.371278,783.712776,3.052634
10,奈良県,18502,15517,149,8148.22,2021-10-14,19,0,75.0,0.187500,0.434629,0.900000,1353837,114.614979,1146.149795,5.539810
28,沖縄県,29986,49980,356,24041.48,2021-10-14,33,6,159.0,0.031250,0.434376,1.062500,1481547,337.350081,3373.500807,10.732025
21,広島県,21692,21884,199,5449.35,2021-10-14,30,0,103.0,0.153846,0.351887,0.875000,2826858,77.414571,774.145712,3.643621
25,新潟県,9570,7963,61,2503.39,2021-10-14,3,0,32.0,2.000000,0.321429,-0.625000,2236042,35.612032,356.120323,1.431100
4,北海道,638,60519,1469,8058.90,2021-10-14,45,0,160.0,1.142857,0.249675,2.214286,5267762,114.885600,1148.856004,3.037343


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
2,佐賀県,26158,5805,29,4981.02,2021-10-14,0,0,20.0,-1.000000,-0.314286,-1.000000,823810,70.465277,704.652772,2.427744
8,大分県,28072,8177,84,5039.67,2021-10-14,1,0,31.0,-0.500000,-0.123170,-0.909091,1151229,71.028440,710.284400,2.692774
42,静岡県,14036,26701,209,5127.13,2021-10-14,2,0,30.0,-0.333333,-0.101984,-0.777778,3708556,71.998373,719.983735,0.808940
5,千葉県,7656,100127,1011,11188.92,2021-10-14,19,1,208.0,-0.472222,-0.080297,-0.558140,6319772,158.434513,1584.345131,3.291258
26,東京都,8294,377036,3069,18948.86,2021-10-14,62,12,540.0,-0.138889,-0.074969,-0.566434,13834925,272.524788,2725.247878,3.903165
32,神奈川県,8932,168585,1294,12818.09,2021-10-14,52,4,397.0,0.040000,-0.068112,-0.490196,9209442,183.056693,1830.566933,4.310793
0,三重県,15312,14725,163,5781.26,2021-10-14,5,0,48.0,-0.285714,-0.042517,-0.666667,1813859,81.180511,811.805107,2.646292
1,京都府,16588,35766,289,9681.17,2021-10-14,9,0,120.0,-0.571429,-0.007986,-0.666667,2545899,140.484756,1404.847561,4.713463
23,愛媛県,24244,5294,82,2753.78,2021-10-14,7,0,68.0,0.000000,0.003721,-0.125000,1369131,38.666862,386.668624,4.966654
18,岡山県,21054,15205,136,5624.71,2021-10-14,3,0,43.0,-0.700000,0.028968,-0.625000,1903627,79.873841,798.738408,2.258846


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
for i,v in enumerate(chart_data["各地の感染者数_1日ごとの発表数"]):
    plt.text(v,i,"{:,}".format(v),va="center",fontsize=7)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
for i,v in enumerate(chart_data["直近1週間の10万人当たりの感染者数"]):
    plt.text(v,i,"{:,}".format(int(v)),va="center",fontsize=7)
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[37  4  3  1  1  0  0  1]
[  1637.     48561.875  95486.75  142411.625 189336.5   236261.375
 283186.25  330111.125 377036.   ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([33.,  4.,  4.,  0.,  3.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 1.,  7., 12.,  9.,  6.,  2.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
22320,2021-10-05,35,山口県,7,5615,1,90,3.46,47.0,0.400000,0.509627,-0.222222
22321,2021-10-06,35,山口県,1,5616,0,90,3.02,41.0,-0.857143,0.418924,-0.857143
22322,2021-10-07,35,山口県,3,5619,0,90,2.43,33.0,2.000000,0.623006,-0.727273
22323,2021-10-08,35,山口県,10,5629,0,90,2.87,39.0,2.333333,1.047248,1.500000
22324,2021-10-09,35,山口県,13,5642,0,90,3.68,50.0,0.300000,1.161534,5.500000
22325,2021-10-10,35,山口県,3,5645,0,90,3.09,42.0,-0.769231,0.408786,-0.727273
22326,2021-10-11,35,山口県,1,5646,1,91,2.80,38.0,-0.666667,0.391470,-0.800000
22327,2021-10-12,35,山口県,13,5659,0,91,3.24,44.0,12.000000,2.048613,0.857143
22328,2021-10-13,35,山口県,10,5669,0,91,3.90,53.0,-0.230769,2.138095,9.000000
22329,2021-10-14,35,山口県,1,5670,1,92,3.76,51.0,-0.900000,1.723810,-0.666667
